In [198]:
import os
if not 'notebooks' in os.listdir():
    os.chdir('..')

In [199]:
print('ok')

ok


In [200]:
import matplotlib.pyplot as plt

In [201]:
from asdfghjkl.operations import Conv2dAug

In [202]:
def plot(x, G=1):
    v = x.reshape(x.shape[0], G, -1, *x.shape[2:])
    
    plt.figure(figsize=(4*G, 4*v.shape[2]))
    for c in range(v.shape[2]):
        for g in range(G):
            plt.subplot(v.shape[2], G, 1 + c * G + g)
            plt.title(f'c: {c}, g: {g}')

            plt.axis('off')
            plt.imshow(v[0, g, c].detach())
    plt.show()

In [250]:
from sparse.rpp_conv_group import D_GConvFull

In [251]:
# Check translation equivariance
B, C, H, W = 32, 3, 32, 32

torch.manual_seed(5)

p = torch.randn((B, C, 10, 10))

x1 = torch.zeros((B, C, H, W))

a1, a2 = 4, 6
x1[:, :, a1:a1+10, a2:a2+10] = p
x1[:, 1, a1:a1+4, a2:a2+4] = 0
x1[:, 1, a1:a1+2, a2:a2+2] = 10.0

x2 = torch.rot90(x1, k=1, dims=[-1, -2])

In [252]:
net = D_Conv(3, 10)

In [253]:
y1 = net(x1)
y2 = net(x2)

print(y1 - y2)


tensor([[ 1.0308e-05,  1.8965e-06,  3.9897e-05,  1.0273e-05, -1.6742e-05,
          1.0398e-05, -3.0781e-05, -4.1325e-06,  3.5208e-05,  1.1478e-05],
        [ 8.0175e-06,  4.5681e-06, -6.9236e-06,  9.6994e-06,  4.5456e-06,
          3.9709e-09,  2.3693e-05,  1.9397e-05, -1.1170e-05,  1.5203e-06],
        [ 8.6697e-06,  1.6125e-05, -1.9047e-05,  1.3843e-05, -1.3890e-05,
         -6.1195e-06,  1.2977e-05,  9.3054e-06,  4.6489e-06,  5.1861e-07],
        [ 1.2108e-07, -2.2213e-06, -6.2093e-06,  8.7750e-07,  5.2822e-06,
          2.1330e-06,  1.5551e-05,  4.4314e-06, -2.4449e-06,  4.5618e-06],
        [-2.1578e-06,  7.4575e-06, -5.6863e-06,  1.1277e-06,  1.0778e-05,
          3.9181e-06,  1.9598e-05,  1.4812e-05, -1.4437e-05,  4.6577e-06],
        [ 1.2810e-05, -1.4412e-05,  1.1418e-06,  4.9380e-06,  1.4493e-05,
          1.5802e-06,  9.2372e-06,  1.9164e-06, -3.3459e-05, -3.1639e-05],
        [-7.0426e-06, -2.1668e-05, -2.7425e-05, -1.8012e-05,  2.3164e-05,
          1.9911e-05,  1.1341e-0

In [254]:
class GConv3d(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, G=4, stride=1, padding=1, padding_mode='circular'):
        super(GConv3d, self).__init__()

        self.conv = nn.Conv3d(in_ch, out_ch, kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
        self.G = G
        
    def forward(self, x):
        B, _, H, W = x.shape
        C = x.shape[1] // self.G
        
        x_aug = x.view(B, self.G, C, H, W)
        x_aug = x_aug.transpose(1, 2)
        
        y_aug = self.conv(x_aug)
        y_aug = y_aug.transpose(1, 2)
        y = y_aug.reshape(B, -1, *y_aug.shape[3:])
        
        return y

In [197]:
layer = GConv3d(3, 10, kernel_size=(4, 3, 3), G=4)
print('x1:', x1.shape)
print(layer(lift_p4(x1)).shape)

x1: torch.Size([32, 3, 32, 32])
torch.Size([32, 30, 32, 32])
